In [ ]:
# from sqlalchemy import create_engine
from itertools import combinations
import logging
from random import shuffle

import pandas as pd
# from tqdm import tqdm_notebook as tqdm
from tqdm.notebook import tqdm

from aeda import utils as u


In [ ]:
from imp import reload
reload(logging)
FORMAT = '%(asctime)s %(message)s'
logging.basicConfig(filename='pk-search-mysql.log', level=logging.INFO, format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S')
logger = logging.getLogger(__name__)

In [ ]:
logging.basicConfig()
logging.getLogger('sqlalchemy').setLevel(logging.ERROR)

In [ ]:
configuration = 'mysql-demo'
# configuration = 'sqlserver-sales'
# configuration = 'sqlserver-production'
conn_string = u.get_db_connection_string(configuration)
connection = u.get_db_connection(conn_string)

In [ ]:
conn_string

In [ ]:
# connection = source_engine.connect()

# table_catalog = 'def'
# table_schema = 'world'

table_catalog = conn_string['catalog']
table_schema = conn_string['schema']

sql = """SELECT C.TABLE_CATALOG
                , C.TABLE_SCHEMA
                , C.TABLE_NAME
                , C.COLUMN_NAME
                , C.ORDINAL_POSITION
                , C.DATA_TYPE
            FROM INFORMATION_SCHEMA.COLUMNS AS C INNER JOIN INFORMATION_SCHEMA.TABLES AS T
            ON C.TABLE_CATALOG = T.TABLE_CATALOG
            AND C.TABLE_SCHEMA = T.TABLE_SCHEMA
            AND C.TABLE_NAME = T.TABLE_NAME
            AND T.TABLE_TYPE = 'BASE TABLE'
            AND T.TABLE_CATALOG = '{}'
            AND T.TABLE_SCHEMA = '{}';""".format(table_catalog, table_schema)

df_tables = pd.read_sql_query(sql, connection)

connection.close()

In [ ]:
table_list = []
for _, t in df_tables.iterrows():
    if t.TABLE_NAME not in table_list:
        table_list.append(t.TABLE_NAME)

In [ ]:
shuffle(table_list)
table_list[:10]

In [ ]:
not_include = []

with open('tables_processed.txt', 'r') as f:
    processed = f.read()
    
processed_list = []
for table in processed.split():
    processed_list.append(table)
    

In [ ]:
def get_columns_from_table(owner, table_name):
    columns = []
    for _, r in df_tables[df_tables['TABLE_NAME'] == table_name].iterrows():
        columns.append(r.COLUMN_NAME)
    logger.info('{} has {} columns'.format(table_name, len(columns)))
    return columns

# Global Test Datasets
df_level_1 = pd.DataFrame()
df_level_2 = pd.DataFrame()
df_level_3 = pd.DataFrame()

def clean_test_datasets(table_name):
    global df_level_1, df_level_2, df_level_3
    logger.info('{} Reseting dataframe samples'.format(table_name))
    df_level_1 = pd.DataFrame()
    df_level_2 = pd.DataFrame()
    df_level_3 = pd.DataFrame()
    return

def get_sql_sample(table_name, columns, top_n):
    if conn_string["db_engine"] == 'mysql':
        sql = """select {} from {} LIMIT {};""".format(', '.join([str('`{}`').format(c) for c in columns]), table_name, top_n)
    elif conn_string["db_engine"] == 'mssqlserver':
        sql = """select top {} * from {}.{};""".format(top_n, conn_string['schema'] , table_name)
    # sql = 'SELECT * FROM {}.{} LIMIT {};'.format(table_name, columns, top_n)
    #sql = """select * from {} where ROWNUM <= {}""".format(table_name, top_n)
    #sql = """select {} from {} where ROWNUM <= {}""".format(', '.join(columns), table_name, top_n)
    #sql = """select {} from {} LIMIT {};""".format(', '.join(columns), table_name, top_n)
    
    return sql

def get_df_sql(sql, connection):
    """
    Returns a dataframe with the results of a query.
    Used to execute evaluations of the PKs on a subsample of the data.
    """
    conn_string = u.get_db_connection_string(configuration)
    connection = u.get_db_connection(conn_string)
    df = pd.read_sql_query(sql, connection)
    connection.close()
    return df

def create_testing_datasets(owner, table_name, columns, n1=10_000, n2=100_000, n3=500_000):
    global df_level_1, df_level_2, df_level_3
    
    logger.info('{} Creating a level 1 dataset sample with {:,} rows'.format(table_name, n1))
    sql = get_sql_sample(table_name, columns, n1)
    df_level_1 = get_df_sql(sql, connection) 
    logger.info('{} Level 1 dataset sample created'.format(table_name))
    
    if len(df_level_1) < n1:
        logger.info('{} Level 1 dataset sample  has {:,} rows, no need for more testing datasets'.format(table_name, len(df_level_1)))
        df_level_2 = df_level_1
        df_level_3 = df_level_1
        return 
    else:
        logger.info('{} Creating a level 2 dataset sample with {:,} rows'.format(table_name, n2))
        sql = get_sql_sample(table_name, columns, n2)
        df_level_2 = get_df_sql(sql, connection) 
        logger.info('{} Level 2 dataset sample created'.format(table_name))
        
        if len(df_level_2) < n2:
            logger.info('{} Level 2 dataset sample has {:,} rows, no need for more testing datasets'.format(table_name, len(df_level_2)))
            df_level_3 = df_level_2
            return 
        else:
            logger.info('{} Creating a level 3 dataset sample with {:,} rows'.format(table_name, n3))
            sql = get_sql_sample(table_name, columns, n3)
            df_level_3 = get_df_sql(sql, connection)
            logger.info('{} Level 3 dataset sample created'.format(table_name))
            
            if len(df_level_3) < n3:
                logger.info('{} Level 3 dataset sample has {:,} rows, no need for testing against the database'.format(table_name, len(df_level_3)))
                return 
    return

def get_column_combinations(columns, k = 5):
    """
    Returns possible combinations of the columns.
    """
    results = combinations(columns, k)
    return results

def count_iterable(i):
    """
    Returns the number of elements in an iterable.
    Used to get the number of combinations to test.
    """
    return sum(1 for e in i)

def get_all_candidates_from_df(table_name, columns, number_of_columns, threshold):
    hundred_percent_counter = 0
    all_candidates = []
    
    if not (hundred_percent_counter > 0) and len(columns) > 0 and len(df_level_1) > 0:
        logger.info('{} Searching candidate keys in combinations of {} columns'.format(table_name, number_of_columns))

        candidates_level_1 = []
        candidates_level_2 = []
        candidates_level_3 = []

        tot_combinations = count_iterable(get_column_combinations(columns, number_of_columns))
        column_combinations = get_column_combinations(columns, number_of_columns)

        logger.info('{} Level 1 dataset: searching {:,} candidates on {:,} records'.format(table_name, tot_combinations, df_level_1.shape[0]))
        for item in tqdm(column_combinations, total=tot_combinations, unit='checks'):
            records = len(df_level_1.groupby(list(item)).size().reset_index(name='Freq'))
            if records / df_level_1.shape[0] >= threshold:
                candidates_level_1.append(item)
        logger.info('{} Level 1 dataset: {:,} candidates out of {:,} combinations tested in {:,} records'.format(table_name, len(candidates_level_1), tot_combinations, df_level_1.shape[0]))

        if len(candidates_level_1) > 0:
            logger.info('{} Level 2 dataset: searching {:,} candidates on {:,} records'.format(table_name, len(candidates_level_1), df_level_2.shape[0]))
            candidates_level_2 = []
            for candidate in tqdm(candidates_level_1):
                records_100k = len(df_level_2.groupby(list(candidate)).size().reset_index(name='Freq'))
                if records_100k / df_level_2.shape[0] > threshold:
                    candidates_level_2.append(candidate)
            logger.info('{} Level 2 dataset: {:,} candidates out of {:,} posibilities tested in {:,} records'.format(table_name, len(candidates_level_2), len(candidates_level_1), df_level_2.shape[0]))

        if len(candidates_level_2) > 0:
            logger.info('{} Level 3 dataset: searching {:,} candidates on {:,} records'.format(table_name, len(candidates_level_2), df_level_3.shape[0]))
            candidates_level_3 = []
            for candidate in tqdm(candidates_level_2):
                records_1M = len(df_level_3.groupby(list(candidate)).size().reset_index(name='Freq'))
                if records_1M / df_level_3.shape[0] > threshold:
                    logger.info('{} Candidate: {} Unique: {:,} Percentage: {:.5%}'.format(table_name, candidate, records_1M, records_1M / df_level_3.shape[0]))
                    candidates_level_3.append(candidate)
                    all_candidates.append(candidate)
                    if records_1M / df_level_3.shape[0] == 1:
                        hundred_percent_counter+=1
            logger.info('{} Level 3 dataset: {:,} candidates out of {:,} posibilities tested in {:,} records'.format(table_name, len(candidates_level_3), len(candidates_level_2), df_level_3.shape[0]))
    return all_candidates

def get_sql(fields, table_name, top_n=10_000):
    if isinstance(fields, list) or isinstance(fields, tuple):
        if top_n > 0:
            sql = """select count(distinct concat({})) as n1, count(*) as n2 from (select top {} * from {}) as t;""".format(', '.join(fields), top_n, table_name)
        else:
            if conn_string["db_engine"] == 'mssqlserver':
                if len(fields) > 1:
                    sql = """select count(distinct concat({})) as n1, count(*) as n2 from {}""".format(', '.join(fields), table_name)
                else:
                    sql = """select count(distinct {}) as n1, count(*) as n2 from {}""".format(fields[0], table_name)
                    # sql = """select count(distinct {}) as n1, count(*) as n2 from {}.{}""".format(', '.join([str('[{}]').format(x) for x in fields]), conn_string["schema"], table_name)
            elif conn_string["db_engine"] == 'mysql':
                sql = """select count(distinct {}) as n1, count(*) as n2 from {}.{}""".format(', '.join([str('`{}`').format(x) for x in fields]), conn_string["schema"], table_name)
            else:
                sql = """select count(distinct {}) as n1, count(*) as n2 from {};""".format(', '.join(fields), table_name)
            
            
    else:
        if top_n > 0:
            sql = """select count(distinct {}) as n1, count(*) as n2 from (select top {} * from {}) as t;""".format(fields, top_n, table_name)
        else:
            if conn_string["db_engine"] == 'mssqlserver':
                sql = """select count(distinct {}) as n1, count(*) as n2 from {}.{}""".format(', '.join([str('{}').format(x) for x in fields]), conn_string["schema"] , table_name)
            elif conn_string["db_engine"] == 'mysql':
                sql = """select count(distinct {}) as n1, count(*) as n2 from {}.{}""".format(', '.join([str('`{}`').format(x) for x in fields]), conn_string["schema"] , table_name)
            else:
                sql = """select count(distinct {}) as n1, count(*) as n2 from {};""".format(fields, table_name)
            
            # 
    return sql

def run_query_on_source(query):
    """
    Execute query on source server and returns the result.
    This shold be used only for queries that returns 1 row such as counts or summaries for 
    performance reasons.
    """
    conn_string = u.get_db_connection_string(configuration)
    connection = u.get_db_connection(conn_string)
    df = pd.read_sql_query(query, connection)
    connection.close()
    return df

def get_final_candidates_from_db(table_name, all_candidates, threshold):
    final_candidates = []
    for candidate in tqdm(all_candidates, unit='candidates'):
        sql = get_sql(candidate, table_name, 0)
        rows = run_query_on_source(sql)
        if rows.iloc[0].n1 / rows.iloc[0].n2 > threshold:
            logger.info('{} In all data: {} columns candidate {} with {:.5%} of unique values or {:,} in {:,} records'.format(table_name, len(candidate), candidate, rows.iloc[0].n1 / rows.iloc[0].n2, rows.iloc[0].n1, rows.iloc[0].n2))
            final_candidates.append(candidate)
    return final_candidates

In [ ]:
def get_candidates(owner, table_name, threshold = 0.99999):
    """
    TODO
    - hundred_percent_counter can be 1 after 1M but could not make it in the full table, need to return to processing more 
    columns.
    """
    columns = get_columns_from_table(owner, table_name)
    clean_test_datasets(table_name)
    if len(columns) > 0:
        create_testing_datasets(owner, table_name, columns, n1=100, n2=10_000,n3=100_000)
    
    threshold = threshold
    
    MAX_COLUMNS = 5
    levels_threshold = [0.99999, 0.98]
    all_candidates = []
    final_candidates = []
    
    number_of_columns = 1
    while len(final_candidates) == 0 and number_of_columns <= MAX_COLUMNS and len(columns) > 0 and len(df_level_1) > 0:
        if len(df_level_1) > 0:
            all_candidates = get_all_candidates_from_df(table_name, columns, number_of_columns, threshold)
            final_candidates = get_final_candidates_from_db(table_name, all_candidates, threshold)

            if number_of_columns == MAX_COLUMNS and len(final_candidates) == 0 and threshold > 0.98:
                number_of_columns = 1
                threshold = threshold * 0.99
                logger.info('{} threshold in: {:.5%}'.format(table_name, threshold))
            elif number_of_columns < MAX_COLUMNS and len(final_candidates) == 0:
                number_of_columns +=1
            elif len(all_candidates) > 0 and len(final_candidates) == 0:
                threshold = threshold * 0.99
                logger.info('{} threshold in: {:.5%}'.format(table_name, threshold))
            elif number_of_columns == MAX_COLUMNS and len(final_candidates) == 0:
                logger.info('{} No candidates found, try with {} columns next time'.format(table_name, MAX_COLUMNS + 1))
        else:
            logger.info('{} has {} rows.'.format(table_name, len(df_level_1)))

In [ ]:
#owner is table_schema 
owner = 'def'
owner = conn_string["catalog"]
owner = conn_string["schema"]
for table in tqdm(table_list):
    #print(table)
    if table not in not_include and table not in processed_list:
        try:
            get_candidates(owner, table)
        #except:
        except Exception as e:
            logger.info('{}: Error {}'.format(table, e))
            pass
        with open('tables_processed', 'a+') as f:
            f.write('{}\n'.format(table))